# Módulo 4: Compartilhamento de dados e próximas etapas.


Nesta seção, examinaremos como alinhar genomas montados, realizar análises filogenéticas e interpretar árvores filogenéticas para investigações de surtos.


Como você já realizou antes:

1.) Clique em Arquivo no canto superior esquerdo e selecione salvar uma cópia na unidade.
Suas alterações não serão salvas se você não executar esta etapa.

2.) Clique no nome da pasta de trabalho no canto superior esquerdo e substitua "Cópia de" pelo seu nome completo.

Você enviará o arquivo do notebook baixado como prova de conclusão deste módulo.

# **Parte 1**: Filogenética e interpretação

## Vamos instalar os pacotes de análise necessários.

Para esta parte, usaremos:
**seqtk** para avaliar a qualidade do genoma (https://github.com/lh3/seqtk)
**Mafft** para alinhamento do genoma (https://mafft.cbrc.jp/alignment/software/)
**snp-site** para avaliação rápida da diferença de SNP (https://github.com/sanger-pathogens/snp-sites)
**Fasttree** para construção de árvore filogenética (http://www.microbesonline.org/fasttree/)
**Phylo** do biopython para visualização rápida da árvore (https://biopython.org/wiki/Phylo). *Nota*: existem muitos programas de visualização de árvores, os mais usados ​​são ggtree(R), ete3 (python) e itol (https://itol.embl.de/).

conta microreact (https://microreact.org/), que você pode configurar/fazer login com sua conta do Google.

In [ ]:
!python --version

Python 3.7.15


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda install -c conda-forge biopython

In [ ]:
!conda install -c bioconda mafft snp-sites fasttree seqtk snp-dists


## Baixe o conjunto de dados a ser analisado.

Existem três conjuntos de dados associados a este módulo. Os dois primeiros conjuntos de dados são exemplos nos slides de instrução. O último conjunto de dados é o conjunto de dados de montagem. Analisar os conjuntos de dados de exemplo juntamente com os vídeos de instrução fornecerá a experiência necessária para analisar o conjunto de dados de avaliação.

In [ ]:
!wget https://wcs_data_transfer.cog.sanger.ac.uk/Module4_data_zip.zip
!unzip Module4_data_zip.zip

### Conjunto de dados 1

Primeiro vamos avaliar a qualidade do genoma para ver se os genomas são bons o suficiente para análise. Para SARs_CoV2, estamos interessados ​​principalmente no número de Ns. O pacote que usaremos é o seqtk comp, que fornece muitas estatísticas para um determinado arquivo de sequência no formato fasta.

Formato de saída: chr, comprimento, #A, #C, #G, #T, #2, #3, #4, #CpG, #tv, #ts, #CpG-ts

9ª coluna, #4 é o número de Ns, 4 ambiguidades, ou seja, A, G, T ou C.

Se você separou arquivos fasta, pode ser mais fácil combiná-los. Caso contrário, você terá que  analisar os arquivos do genoma separadamente.

In [ ]:
O comando #cat (concatenar) combina os fastas em um arquivo
#seqtk comp analisa as informações da sequência
#cut -f 1,9 seleciona a coluna 1 e 9, as informações que precisamos.
!cat Module4_data/dataset_1_flight/20CV*.fasta > dataset_1_flight_combined.fasta
!seqtk comp dataset_1_flight_combined.fasta | cut -f 1,9 > dataset_1_quality.tsv

In [ ]:
!cat dataset_1_quality.tsv

A qualidade parece boa, 20CV0410 não é tão boa quanto o resto. Tem 1140 bases ambíguas, mas é bom o suficiente para análise. Em geral, exigimos 90% ou menos de **3000** Ns do genoma a ser montado para análise da árvore filogenética. No entanto, existem grupos que têm requisitos menos rigorosos, alguns chegam a 50%. Ter 70% do genoma montado é o padrão no pangolim para atribuição de linhagem.

### Alinhamento

Usaremos mafft para alinhamento. É bastante rápido e bastante preciso.

Existem muitas opções para alinhar sequências em mafft. --auto é uma boa opção onde o próprio programa escolhe o algoritmo mais eficiente (bom equilíbrio entre velocidade e precisão). O alinhamento pode levar dias para sequências muito longas, se tiver optado o algoritmo mais preciso. Se você estiver alinhando sequências curtas, como uma região do gene/proteína spike, poderá usar opções mais precisas. O mafft tem bons exemplos do que usar e quando usar.

Para mafft alinhar as sequências, no nosso caso, genomas inteiros, você precisa combinar as sequências que deseja alinhar em um arquivo. O bom é que você já fez isso e, como todos os genomas são bons, não precisamos excluir nenhum do alinhamento.

In [ ]:
!mafft --auto dataset_1_flight_combined.fasta > dataset_1_flight_aligned.fasta

### Rápida olhada nas diferenças do genoma
O snp-site é um software realmente bom para lhe dar uma ideia de quão intimamente relacionados estão seus genomas de interesse. Para pequenos conjuntos de dados como este, isso é muito bom. Ele analisa seu alinhamento e fornece um alinhamento SNP como padrão. ele também pode fornecer informações snp no formato VCF.

In [ ]:
!snp-sites dataset_1_flight_aligned.fasta

a partir dessa rápida olhada, você verá que a maioria dos genomas é indistinguível, exceto 20CV0401, que possui um SNP adicional.

### Interpretação

o que podemos dizer sobre um monte de genomas idênticos?

Como sabemos que essas pessoas vieram no mesmo voo, podemos analisar um dos genomas usando https://genome.ucsc.edu/cgi-bin/hgPhyloPlace ou se você tiver uma conta, pode usar o programa GISAID audacity para encontrar o genoma internacional mais próximo. Baixe um genoma e tente isso com seu navegador da web.

consulte a publicação https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7920679/ para a análise deste conjunto de dados.

In [ ]:
!cat Module4_data/dataset_1_flight/dataset_1_flight_metadata.tsv

### Dataset 2

In [ ]:
!seqtk comp Module4_data/dataset_2_hotel/dataset_2_quarantineHotel.fasta | cut -f 1,9 > dataset_2_quality.tsv
!cat dataset_2_quality.tsv
#as montagens do genoma são de boa qualidade?

In [ ]:
#Os arquivos fasta já estão concatenados.
!mafft --auto Module4_data/dataset_2_hotel/dataset_2_quarantineHotel.fasta > dataset_2_quarantineHotel_aligned.fasta

In [ ]:
!snp-sites dataset_2_quarantineHotel_aligned.fasta

In [ ]:
#pode ver a distância snp entre pares de amostras usando snp-dist
!snp-dists dataset_2_quarantineHotel_aligned.fasta


O alinhamento é um pouco mais complicado com mais diferenças, precisamos de uma árvore filogenética para visualizar as relações

In [ ]:
!fasttree -nt -gtr -gamma dataset_2_quarantineHotel_aligned.fasta > dataset_2_quarantineHotel_aligned.nwk

In [ ]:
!cat dataset_2_quarantineHotel_aligned.nwk
#como você pode ver é muito difícil interpretar este formato sem visualizar em forma de árvore

In [ ]:
from Bio import Phylo

tree = Phylo.read("dataset_2_quarantineHotel_aligned.nwk", "newick")
print("tree")
Phylo.draw_ascii(tree)


### Interpretação

Este conjunto de dados é sobre uma incursão e disseminação para a comunidade a partir do hotel de quarentena. O que podemos dizer sobre essa árvore? Acompanhe a publicação e o vídeo de instruções. Metadados associados ao caso em dataset_2_hotel/dataset_2_quarantineHotel_metadata.tsv.


Consulte a publicação https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8084504/ para a análise deste conjunto de dados.

Compare a árvore acima com a publicação, por que parece diferente?

In [ ]:
!cat Module4_data/dataset_2_hotel/dataset_2_quarantineHotel_metadata.tsv

In [ ]:
#Vamos adicionar o genoma de referência de 2020 ao conjunto de dados.
!mafft --auto --add Module4_data/dataset_2_hotel/dataset_2_quarantineHotel.fasta Module4_data/dataset_2_hotel/references.fasta > dataset_2_quarantineHotel_aligned_ref.fasta

In [ ]:
!fasttree -nt -gtr -gamma dataset_2_quarantineHotel_aligned_ref.fasta > dataset_2_quarantineHotel_aligned_ref.nwk

Agora, se enraizarmos a filogenia usando a referência, a árvore se parecerá mais com a publicação. Por quê?

Dê uma olhada no alinhamento snp também usando snp-site. Você verá que o uso da referência permitiu a identificação de bases ancestrais, então agora sabemos que mudanças de nucleotídeos são mutações de tipo selvagem. Anteriormente, o algoritmo da árvore fazia a suposição de que a base majoritária é a sequência ancestral, o que não é correto.

In [ ]:
from Bio import Phylo

treeOutgroup = Phylo.read("dataset_2_quarantineHotel_aligned_ref.nwk", "newick")
treeOutgroup.root_with_outgroup({"name": "nCoV2019|Wuhan-Hu-1|MN908947|China|Wuhan|2019-12"}) 
print("treeOutgroup")
Phylo.draw_ascii(treeOutgroup)

## Assista a demonstração/vídeo micoreact e nextstrain

Olhar para as árvores dessa maneira é muito difícil, existem algumas boas ferramentas por aí que permitem árvores interativas e visualização de metadados.

https://docs.microreact.org/
https://nextstrain.org/community/narratives/ESR-NZ/GenomicsNarrativeSARSCoV2/aotearoa-border-incursions

Projeto de demonstração de microrreação do conjunto de dados 2
https://microreact.org/project/5ELv2rXSKKeZ8XZCFXq9Ug-dataset2hotel

### Outras ferramentas para conjuntos de dados maiores e construção de árvores
Mafft, mesmo com o modelo de precisão mínima, pode ser bastante lento quando você tem milhares de genomas para alinhar. Para acelerar as coisas, você pode adicionar novos genomas a alinhamentos existentes ou dividir genomas em grupos como linhagens ou sublinhagens e, em seguida, mesclar os alinhamentos. O  mafft tem uma boa explicação desses truques e seus problemas.

Nextalign é outra ferramenta que você pode usar. É super rápido! (https://docs.nextstrain.org/projects/nextclade/en/stable/user/nextalign-cli.html)

igtree2 é um pacote de construção de filogenia abrangente (não baysiano) que pode testar os melhores modelos de substituição para o seu conjunto de dados, realizar análises de bootstrap, comparar filogenia etc etc. . (http://www.iqtree.org/)

usher é um pacote de software que permite adicionar sequências a árvores existentes sem fazer todo o alinhamento novamente. (https://www.nature.com/articles/s41588-021-00862-7) Usamos isso quando analisamos o conjunto de dados de voo. Existem algumas advertências e problemas dos quais você precisa estar ciente. Se o alinhamento existente for realmente limpo e as sequências a serem adicionadas forem de alta qualidade, usar o usher é uma boa maneira rápida de obter alguns resultados preliminares rápidos. No entanto, ele não lida muito bem com Ns ou lacunas.

Para fiolgenia baysiana cronometrada, BEAST2 é frequentemente usado. (https://www.beast2.org/) Aqui está uma boa revisão. https://www.nature.com/articles/s41559-017-0280-x

### Agora estamos prontos para a tarefa

As células abaixo serão verificadas.

Para esta tarefa, você investigará um possível surto em uma unidade de saúde. Os genomas incluídos são de casos que ocorreram na época do surto e todos têm alguns links para o estabelecimento de saúde. Alguns casos tiveram contatos claros com outros casos conhecidos (Epi-Links com Surtos: sim) e para alguns contatos não foram encontrados (Epi-links com Surtos: não encontrados). E alguns casos são desconhecidos (Epi-links com surtos: desconhecido). Seu trabalho é usar a árvore e a data do relatório para decidir se os casos rotulados como desconhecidos devem ser mais investigados, se estão geneticamente ligados ao surto ou podem ser excluídos do surto.

Você precisa:
1) Controlar os genomas fornecidos e excluir quaisquer genomas de baixa qualidade da análise
2) Alinhar os genomas de boa qualidade (**<3000 Ns**) e construir uma árvore filogenética
3) Use a árvore que você gerou e os metadados fornecidos para configurar um projeto de microrreação para visualização e exame da árvore
4) Usando os metadados fornecidos para decidir se os isolados em questão estão no surto
5) Responda às perguntas da avaliação

**dicas**

você pode usar seqtk subseq para remover sequências de baixa qualidade ou pode mover (mv) as sequências de baixa qualidade para outro diretório.

Use a distância snp para ver se alguns dos casos são "muito" diferentes do resto.

In [ ]:
# Controle de qualidade dos conjuntos de genoma para ver se algum deles não é adequado para análise posterior.

In [ ]:
#liste abaixo os isolados que você acha que podem não ser adequados para análise futura
#alterar XXX para isolar numero
!echo "esses isolados não atendem aos padrões mínimos de qualidade: XXX e XXX"

In [ ]:
#Alinhar os genomas de boa qualidade com mafft

In [ ]:
#Construir uma árvore filogenética com o alinhamento mafft

In [ ]:
#Visualize a árvore com Phylo
#Lembre-se de "from Bio import Phylo" se ainda não o fez

Agora você tem todos os arquivos necessários para criar um projeto microreact.
Baixe sua árvore e o arquivo de metadados: surto_microreact.tsv

**Carregue uma captura de tela do seu projeto de microrreação na célula abaixo**
Você pode fazer isso clicando duas vezes na célula abaixo
em seguida, clique no ícone de inserção de imagem, preto com montanha branca e selecione o arquivo iamge. Dependendo da imagem pode ser um pouco lento.
A imagem deve ser salva assim que você salvar seu notebook.

**Insira sua captura de tela microreact aqui**
clique duas vezes aqui.

**Agora que você tem o projeto microreact, será mais fácil responder às perguntas de avaliação**

Opcional, use snp-dist e snp-site para ajudá-lo na análise

**Salve seu caderno agora**

## **Parte 2** Compartilhamento de dados

Para fazer a parte 2 do módulo 4
siga os vídeos e os slides  nas **Seções 5, 6 e 7** 